In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [ ]:
model = joblib.load('xgb_model.pkl')
scaler = joblib.load('scaler.pkl')

In [ ]:
input_array = np.array([1,1,1,1,1,1,1,0,1,1,25.06830214,1,1,1,1,1,0,13.99540321,3,1])

# These are the original column names used during training (ensure they are correct)
columns = ['ID','A1_Score','A2_Score','A3_Score','A4_Score','A5_Score','A6_Score','A7_Score','A8_Score','A9_Score','A10_Score','age','gender','ethnicity','jaundice','contry_of_res','result','age_desc','relation','Class/ASD']  # Replace with actual column names

# Convert input array to a DataFrame with the same column names
expected_features = [
    'A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 
    'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 
    'A9_Score', 'A10_Score', 'age', 'gender', 
    'ethnicity', 'jaundice', 'austim', 'contry_of_res', 
    'used_app_before', 'sum_score', 'ind', 
    'ageGroup']

input_df = pd.DataFrame([input_array], columns=columns)
input_df['austim'] = 0  # Default value
input_df['used_app_before'] = 0  # Default value
input_df['age_desc'] = 0  # Default value or a suitable placeholder

def convert_age(age):
    
    if age < 4:
        return 'Toddler'
    elif age < 12:
        return 'Kid'
    elif age < 18:
        return 'Teenager'
    elif age < 40:
        return 'Young'
    else:
        return 'Senior'
        

input_df['ageGroup'] = input_df['age'].apply(convert_age)

input_df = pd.get_dummies(input_df, columns=['ageGroup'], drop_first=True)

for feature in expected_features:
    if feature not in input_df.columns:
        input_df[feature] = 0 
# Recreate `sum_score`
input_df['sum_score'] = input_df[['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 
                                  'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']].sum(axis=1)

 #Recreate `ind` (using default values for 'austim' and 'used_app_before')
input_df['ind'] = input_df['jaundice'] + input_df['austim'] + input_df['used_app_before']

# Drop unnecessary columns (if not needed for the model)
input_df = input_df.drop(columns=['ID','austim', 'used_app_before', 'age_desc','Class/ASD'])

# Scale the input DataFrame
input_df_scaled = scaler.transform(input_df)

In [ ]:
prediction = model.predict(input_df_scaled)


In [ ]:
print("Prediction: ", prediction)
